## Initialize Fiddler Client


In [ ]:
import fiddler as fdl
client = fdl.FiddlerApi()

## Load prediction log

Prediction logs must contains the models input features and predictions.

In [ ]:
import pandas as pd
prediction_log_df = pd.read_csv('/app/fiddler_samples/samples/datasets/p2p_loans/p2p_production.log')
prediction_log_df.head(3)

## setup/cleanup project

In [ ]:
project_id = 'tutorial'
model_id = 'no_model_artifact'

if project_id in client.list_projects():
    client.delete_model(project_id, model_id)
    client.delete_dataset(model_id)
else:
    client.create_project(project_id)

## Create model from prediction log

In [ ]:
outputs = ['probability_fully_paid']

is_feature = lambda x: x not in outputs
features = list(filter(is_feature, list(prediction_log_df.columns)))

client.create_model_from_prediction_log(
    project_id,
    model_id,
    prediction_log_df,
    features,
    outputs,
    model_task=fdl.ModelTask.BINARY_CLASSIFICATION,
)

## Send monitoring events

In [ ]:
import datetime
import random
from IPython.display import clear_output
import time

two_weeks_ms = 604800 * 2 * 1000

chunked_df = [prediction_log_df[i:i+1] for i in range(0, prediction_log_df.shape[0], 1)]
count = 0
for row_df in chunked_df:
    data = row_df.to_dict('records')[0]
    occurred_at = int(round(time.time() * 1000)) - random.randint(0, two_weeks_ms)

    result = client.publish_event(project_id, model_id, data, event_time_stamp=occurred_at)
    
    count += 1
    clear_output(wait = True)
    print(f'Sending {count} / {len(prediction_log_df)}')
    time.sleep(2)